# 실습 7: 웹 애플리케이션 연동

## 목표
- Flask API 서버 생성
- Frontend 설정
- 전체 시스템 테스트

## 7-1. 환경 변수 로드

In [ ]:
from dotenv import load_dotenv
import os

load_dotenv()

REGION = os.getenv('AWS_REGION', 'us-west-2')
WINDY_API_KEY = os.getenv('WINDY_API_KEY', '')
AGENT_RUNTIME_ARN = os.getenv('AGENT_RUNTIME_ARN', '')

print(f"🌍 Region: {REGION}")
print(f"🌤️ Windy API Key: {WINDY_API_KEY if WINDY_API_KEY else '미설정'}")
print(f"🤖 Agent ARN: {AGENT_RUNTIME_ARN if AGENT_RUNTIME_ARN else '미설정'}")

## 5-2. Flask 서버 코드 생성

In [ ]:
import os

# main.py 내용 생성 (.env 값 사용)
main_content = f'''import boto3
import json
import re
from flask import Flask, request, Response, stream_with_context
from flask_cors import CORS
from botocore.config import Config

app = Flask(__name__)
CORS(app)

config = Config(
    read_timeout=3000,
    connect_timeout=600,
    retries={{'max_attempts': 0}}
)

client = boto3.client('bedrock-agentcore', region_name='{REGION}', config=config)

AGENT_RUNTIME_ARN = '{AGENT_RUNTIME_ARN}'

@app.route('/analyze', methods=['POST'])
def analyze():
    data = request.json
    prompt = data.get('prompt', '')
    actor_id = data.get('actor_id', 'default-user')
    session_id = data.get('session_id', 'default-session')
    
    payload = json.dumps({{
        "prompt": prompt,
        "actor_id": actor_id,
        "session_id": session_id
    }}).encode()
    
    response = client.invoke_agent_runtime(
        agentRuntimeArn=AGENT_RUNTIME_ARN,
        runtimeSessionId='workshop-session-' + session_id + '-' + str(hash(actor_id))[:10],
        payload=payload
    )
    
    def generate():
        if "text/event-stream" in response.get("contentType", ""):
            for line in response["response"].iter_lines(chunk_size=10):
                if line:
                    line = line.decode("utf-8")
                    if line.startswith("data: "):
                        match = re.search(r"'data':\\s*'([^']*)'" , line)
                        if match:
                            yield match.group(1)
    
    return Response(stream_with_context(generate()), content_type='text/plain; charset=utf-8')

if __name__ == '__main__':
    print("🚀 Flask 서버 시작...")
    print(f"📍 http://localhost:8082")
    app.run(host='0.0.0.0', port=8082, debug=True)
'''

# 디렉토리 생성
os.makedirs('agent', exist_ok=True)

# main.py 파일 저장
with open('agent/main.py', 'w', encoding='utf-8') as f:
    f.write(main_content)

print("✅ agent/main.py 생성 완료")
print(f"📝 Agent ARN: {AGENT_RUNTIME_ARN}")

## 5-3. env.js 파일 업데이트

In [ ]:
import os

# SageMaker Studio 도메인 입력 (수동)
print("현재 Jupyter URL을 확인하세요.")
print("\n아래에 Studio 도메인을 입력하세요 (예:https://njbo9fhsmle8erv.studio.us-west-2.sagemaker.aws) ")

STUDIO_DOMAIN = input("Studio 도메인: ").strip()

# AGENT_API_URL 결정
# SageMaker Studio
agent_api_url = f'{STUDIO_DOMAIN}/jupyterlab/default/proxy/8082/analyze'
print(f"\n✅ SageMaker Studio 모드")

env_js_content = f"""const ENV = {{
    USER_ID: 'workshop-user',
    SESSION_ID: 'workshop-session',
    WINDY_API_KEY: '{WINDY_API_KEY}',
    WINDY_API_URL: 'https://api.windy.com/api/point-forecast/v2',
    AGENT_API_URL: '{agent_api_url}',
}};
"""

with open('env.js', 'w') as f:
    f.write(env_js_content)

print("\n✅ env.js 파일 생성 완료")
print(f"📝 AGENT_API_URL: {agent_api_url}")

## 5-4. 간단한 HTTP 서버 스크립트 생성

SageMaker Studio에서는 로컬 HTML 파일을 직접 열 수 없으므로 HTTP 서버가 필요합니다.

In [ ]:
http_server_script = '''#!/bin/bash
echo "🌐 HTTP 서버 시작 (포트 8000)..."
python3 -m http.server 8000
'''

with open('start_http_server.sh', 'w') as f:
    f.write(http_server_script)

import os
os.chmod('start_http_server.sh', 0o755)

print("✅ start_http_server.sh 생성 완료")

## 5-5. 서버 실행 방법

In [ ]:
%%writefile start_server.sh
#!/bin/bash
echo "🚀 Flask 서버 시작..."
cd agent
python main.py

In [ ]:
# 실행 권한 부여
!chmod +x start_server.sh
print("✅ start_server.sh 생성 완료")

## 5-5. 서버 실행 방법

### 터미널 1: Flask 서버 (Backend)
```bash
./start_server.sh
# 또는
cd agent && python main.py
```

### 터미널 2: HTTP 서버 (Frontend)
```bash
./start_http_server.sh
# 또는
python3 -m http.server 8000
```

### 접근 주소

**로컬 환경:**
- Frontend: http://localhost:8000
- Backend: http://localhost:8082

**SageMaker Studio:**
- Frontend: `https://{studio-domain}/jupyterlab/default/proxy/8000/`
- Backend: `https://{studio-domain}/jupyterlab/default/proxy/8082/`

예시:
```
https://njbo9fhsmle8erv.studio.us-west-2.sagemaker.aws/jupyter/default/proxy/8000/
```

## 5-6. Frontend 테스트

### 🚀 실행 순서

#### 1. 두 개의 터미널 열기

**터미널 1 (Backend):**
```bash
cd agent
python main.py
```

**터미널 2 (Frontend):**
```bash
python3 -m http.server 8000
```

#### 2. 브라우저에서 접근

**로컬 환경:**
```
http://localhost:8000
```

**SageMaker Studio:**
```
https://njbo9fhsmle8erv.studio.us-west-2.sagemaker.aws/jupyter/default/proxy/8000/
```

패턴:
```
https://{your-studio-domain}/jupyter/default/proxy/8000/
```

### 🔧 SageMaker Studio 추가 설정

**env.js 파일 수정 필요:**
```javascript
const ENV = {
    AGENT_API_URL: 'https://njbo9fhsmle8erv.studio.us-west-2.sagemaker.aws/jupyter/default/proxy/8082/analyze',
    // 위 주소를 본인의 Studio 도메인으로 변경
};
```

### 테스트 쿼리
```
서울특별시 서초구 방배중앙로 06681에서 화재가 발생했습니다. 
가까운 소방서를 찾아주세요.
```

### 예상 결과
- ✅ 가까운 소방서 5곳 표시
- ✅ 지도에 마커 표시
- ✅ 기상 정보 (풍속, 풍향, 온도)
- ✅ 관련 뉴스

## 5-7. 서버 상태 확인

In [ ]:
import requests
import time

# 서버가 실행 중인지 확인
try:
    time.sleep(2)
    response = requests.get('http://localhost:8082', timeout=5)
    print("✅ Flask 서버 실행 중")
except:
    print("⚠️ Flask 서버가 실행되지 않았습니다")
    print("👉 터미널에서 ./start_server.sh 실행하세요")

## 5-8. 전체 시스템 요약

In [ ]:
print("="*60)
print("🎉 워크샵 완료!")
print("="*60)
print("\n📋 생성된 리소스:")
print(f"   - Memory ID: {os.environ.get('MEMORY_ID', 'N/A')}")
print(f"   - S3 Bucket: firestation-location-xy")
print(f"   - Vector Index: fire-station")
print(f"   - Agent ARN: {AGENT_RUNTIME_ARN}")
print("\n📁 생성된 파일:")
print("   - agent/main.py (Flask 서버)")
print("   - agent/deploy/runtime/config.py")
print("   - env.js")
print("   - start_server.sh")
print("\n🚀 실행 방법:")
print("   1. 터미널: ./start_server.sh")
print("   2. 브라우저: index.html 열기")
print("   3. 테스트 쿼리 입력")
print("\n📍 서버 주소: http://localhost:8082")
print("="*60)

## ✅ 실습 5 완료!

### 전체 워크샵 완료! 🎉

### 구축한 시스템
1. **Memory 시스템** (Short-term & Long-term)
2. **Vector Database** (S3 Vectors)
3. **Agent Runtime** (Bedrock Agent Core)
4. **Tools** (소방서 검색, 기상 정보, 뉴스 검색)
5. **Web Application** (Flask + Frontend)

### 다음 단계
- 다양한 시나리오 테스트
- Memory 기능 확인
- 추가 기능 개발